# Lesson 3 Jupyter Notebook Companion

In [15]:
# Execute a command that has a cell output
print("Hello Jupyter")

Hello Jupyter


In [31]:
# Execute a command that raises an error
print("Hello Jupyter", end=1)

TypeError: end must be None or a string, not int

# Recap of lesson 2

We are following the exercise available in [lesson 2 of the python introduction for PhD IMCBio course](https://vgilbart.github.io/python-intro/lecture/lesson-2.html#exercise-2).

The dataset studied describes the **percent of A, T, C, G** in 5 sequences:
- seq1
- seq2
- seq3
- seq4
- seq5


## Setup


We import the dataset as a pandas datatable and reformat it.

In [16]:
import pandas as pd

df = pd.read_csv('exercise/data/example.txt', index_col=0, sep=' ')
df = pd.melt(df, var_name='nucl', value_name='freq', ignore_index=False)

## Description of dataset

Here are some basic information about the dataset to understand its content.

In [17]:
df.index

Index(['seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq1', 'seq2', 'seq3', 'seq4',
       'seq5', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq1', 'seq2', 'seq3',
       'seq4', 'seq5'],
      dtype='object', name='Seq')

In [18]:
df.columns

Index(['nucl', 'freq'], dtype='object')

In [19]:
df.head()

,nucl,freq
Seq,,
seq1,A,0.46
seq2,A,0.20
seq3,A,0.16
seq4,A,0.18
seq5,A,0.26


## Statistics

We will calculate the mean value frequencies of all nucleotides.

In [20]:
df['freq'].mean()

0.25

We will calculate the mean value frequencies per nucleotides.

In [21]:
df.groupby("nucl")[['freq']].mean()

,freq
nucl,
A,0.252
C,0.252
G,0.256
T,0.240


# Testing magic commands

In [23]:
# %load exercise/script/analyse_fasta.py
#!/usr/bin/env python3

import sys 

def nucl_freq(seq):
  if not isinstance(seq, str):
    raise TypeError("Input must be a string.")
  valid_nucl = {"A", "T", "C", "G"}
  seq_nucl = set(seq)
  if seq_nucl.difference(valid_nucl) != set():
    raise ValueError("Input string must only contain characters A, C, T or G.")
    
  n = len(seq)
  freq = dict()
  for nucl in valid_nucl:
    freq[nucl] = seq.count(nucl)/n
  return freq 

def analyse_fasta(input_file, output_file):
  freq = {}
  
  with open(input_file, 'r') as input:
    for line in input:
      if line.startswith(">"):
        sequence_name = line.strip()[1:]
      else: 
        current_sequence = line.strip()
        freq[sequence_name] = nucl_freq(current_sequence)

  with open(output_file, 'w') as output:
    output.write("Seq A T C G\n")
    for key, value in freq.items():
      output.write(f"{key} {value.get('A')} {value.get('T')} {value.get('C')} {value.get('G')}\n")

  return None


In [24]:
nucl_freq("AACTTG")

{'T': 0.3333333333333333,
 'C': 0.16666666666666666,
 'G': 0.16666666666666666,
 'A': 0.3333333333333333}

In [25]:
%whos

del nucl_freq, analyse_fasta

%whos


Variable           Type                Data/Info
------------------------------------------------
analyse_fasta      function            <function analyse_fasta at 0x132eb7100>
f                  function            <function f at 0x10589a840>
fig                Figure              Figure({\n    'data': [{'<...>': {'text': 'freq'}}}\n})
interact           _InteractFactory    <ipywidgets.widgets.inter<...>ry object at 0x105887a50>
interactive_plot   function            <function interactive_plot at 0x113baccc0>
nucl_freq          function            <function nucl_freq at 0x132eb7740>
pd                 module              <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
plt                module              <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
px                 module              <module 'plotly.express' <...>tly/express/__init__.py'>
sys                module              <module 'sys' (built-in)>
widgets            module              <module 'ipywidgets.

In [26]:
%run exercise/script/analyse_fasta.py

nucl_freq("AACTTG")

{'T': 0.3333333333333333,
 'C': 0.16666666666666666,
 'G': 0.16666666666666666,
 'A': 0.3333333333333333}

# Interactive plots

## Widgets

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import widgets

# Create a DataFrame from the given data
df = pd.read_csv('exercise/data/example.txt', index_col=0, sep=' ')

def interactive_plot(columns):
    # Plot the data
    fig, ax = plt.subplots(1, figsize=(5, 6))

    bottom = pd.Series([0,0,0,0,0], index = df.index) 
    for col in columns:
        ax.bar(df.index, df[col], label=col, bottom=bottom)
        bottom += df[col]

    # Add labels and legend
    ax.set_xlabel('Sequences')
    ax.set_ylabel('Frequency')
    ax.legend(title='Nucleotides', bbox_to_anchor=(0, 1),
              loc='lower left', ncols=4)
    plt.show()

In [4]:
interact(interactive_plot, columns=df.columns)

interactive(children=(Dropdown(description='columns', options=('A', 'T', 'C', 'G'), value='A'), Output()), _do…

<function __main__.interactive_plot(columns)>

In [5]:
interact(interactive_plot, columns=widgets.SelectMultiple(
    options=df.columns,
    value=[df.columns[0]],
    description='Fruits',
    disabled=False
))

interactive(children=(SelectMultiple(description='Fruits', index=(0,), options=('A', 'T', 'C', 'G'), value=('A…

<function __main__.interactive_plot(columns)>

## Plotly express

In [2]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('exercise/data/example.txt', sep=' ')
df = pd.melt(df, var_name='nucl', value_name='freq', id_vars=['Seq'])

fig = px.bar(df, x="Seq", y="freq", color = "nucl", title="Nucleotide frequency")
fig.show()